In [3]:
def count_repeat(seq, num_repeats=8):
    count_repeat_A = "A" * num_repeats
    count_repeat_T = "T" * num_repeats
    count_repeat_G = "G" * num_repeats
    count_repeat_C = "C" * num_repeats
    if count_repeat_A in seq or count_repeat_T in seq or count_repeat_G in seq or count_repeat_C in seq:
        return True

In [4]:
#test

seq1 = "CCCCCCCCCCCCC"
seq2 = "AAAAA"
    
count_repeat(seq1), count_repeat(seq2)

(True, None)

In [5]:
count_repeat(seq2)

In [6]:
#test
trial = [('AAAAAAAAAA', 'ATGCATGCAT'),
  ('AAAAAAAAAT', 'TGCATGCATG'),
  ('AAAAAAAATG', 'GCATGCATGC'),
  ('AAAAAAATGC', 'CATGCATGCA'),
  ('AAAAAATGCA', 'ATGCATGCAT'),
  ('AAAAATGCAT', 'AAAAAAAAAA'),
  ('AAAATGCATG', 'GCATGCATGC')]

for (i,j) in trial:
    trail2 = []
    if count_repeat(i) != True and count_repeat(j) != True:
        trail2.append((i,j))
        print(trail2)

[('AAAAAAATGC', 'CATGCATGCA')]
[('AAAAAATGCA', 'ATGCATGCAT')]
[('AAAATGCATG', 'GCATGCATGC')]


In [7]:
def probes(seq, extension_probe_length, ligation_probe_length, insert_size):

   
    # extension_probe_length = 18bp (ideally: 16-20bp)
    # ligation_probe_length = 22bp (ideally: 20-24bp)
    # insert_size = 112 nts
    extension_probe = []
    ligation_probe = []
    
    #all positions are zero-based
    extension_probe_pos =[]
    ligation_probe_pos = []
    target_seq = [] 

    # selecting probes of defined length iteratively from 5'-3' of the input sequence 
    for i in range(len(seq)):
        for j in range(i + 1, len(seq) + 1):
            if len(seq[i:j]) == extension_probe_length and len(seq[j+insert_size:j+insert_size+ligation_probe_length]) == ligation_probe_length:
                # discard sequences with consecutive polyNs (=repeats)
                if count_repeat(seq[i:j]) == True or count_repeat(seq[j+insert_size:j+insert_size+ligation_probe_length]) == True:
                    break
                
                # if sequences do not contain consecutive polyNs, proceed
                else:
                    # extract extension_probe of required length and store in a list
                    extension_probe.append(seq[i:j])
                    
                    # note start and end indexes of extension_probe 
                    extension_probe_pos.append(f"start={i} end={j}")
        
                    
                    # skip 112 nts (=insert size) and extract following ligation_probe of required length
                    ligation_probe.append(seq[j+insert_size:j+insert_size+ligation_probe_length])
                    
                    # note start and end indexes of extension_probe and store in a list
                    ligation_probe_pos.append(f"start={j+insert_size} end={j+insert_size+ligation_probe_length}")
                    
                      
                
                
    all_probes = list(zip(extension_probe,extension_probe_pos, ligation_probe, ligation_probe_pos)) # combine lists 
    all_probes_only = list(zip(extension_probe, ligation_probe))       
    return all_probes, extension_probe_pos, ligation_probe_pos, all_probes_only


def check_probes(all_probes): #using all_probes as list
    import Bio.SeqUtils 
    from Bio.SeqUtils import MeltingTemp as tm
    
    new_probes = []
    final_probes = []
    melting_temp = []
    
    # check for GC content of all_probes (expected range: 40-70%)
    for (i,j) in all_probes:
        if Bio.SeqUtils.GC(i) >= 40 or Bio.SeqUtils.GC(i) <= 70 and Bio.SeqUtils.GC(i) >= 40 or Bio.SeqUtils.GC(i) <= 70:
            # keep only those which follows the above criteria
            new_probes.append((i,j))
            
    # check for tm = melting temperature of all_probes 
    for (i,j) in new_probes:
        if tm.Tm_Wallace(i) >= 60 and tm.Tm_Wallace(j) >= 60:
            # keep only those which follows the above criteria
            final_probes.append((i,j))
            melting_temp.append(tm.Tm_Wallace(i))
            melting_temp.append(tm.Tm_Wallace(j))
                
    return final_probes
    

In [8]:
seq = 'ATGTCACTGTATCGCCCTCTAGTTCTGCTGTCTTGTCTCTCATGGCCGCTGGCTGGCTTTTCTGCCACCGCGCTGACCAACCTCGTCGCGGAACCATTCGCTAAACTCGAACAGGACTTTGGCGGCTCCATCGGTGTGTACGCGATGGATACCGGCTCAGGCGCAACTGTAAGTTACCGCGCTGAGGAGCGCTTCCCACTGTGCAGCTCATTCAAGGGCTTTCTTGCTGCCGCTGTGCTGGCTCGCAGCCAGCAGCAGGCCGGCTTGCTGGACACACCCATCCGTTACGGCAAAAATGCGCTGGTTCCGTGGTCACCCATCTCGGAAAAATATCTGACAACAGGCATGACGGTGGCGGAGCTGTCCGCGGCCGCCGTGCAATACAGTGATAACGCCGCCGCCAATTTGTTGCTGAAGGAGTTGGGCGGCCCGGCCGGGCTGACGGCCTTCATGCGCTCTATCGGCGATACCACGTTCCGTCTGGACCGCTGGGAGCTGGAGCTGAACTCCGCCATCCCAGGCGATGCGCGCGATACCTCATCGCCGCGCGCCGTGACGGAAAGCTTACAAAAACTGACACTGGGCTCTGCACTGGCTGCGCCGCAGCGGCAGCAGTTTGTTGATTGGCTAAAGGGAAACACGACCGGCAACCACCGCATCCGCGCGGCGGTGCCGGCAGACTGGGCAGTCGGAGACAAAACCGGAACCTGCGGAGTGTATGGCACGGCAAATGACTATGCCGTCGTCTGGCCCACTGGGCGCGCACCTATTGTGTTGGCCGTCTACACCCGGGCGCCTAACAAGGATGACAAGCACAGCGAGGCCGTCATCGCCGCTGCGGCTAGACTCGCGCTCGAGGGATTGGGCGTCAACGGGCAGTAA'

In [9]:
all_probes, extension_probe_pos, ligation_probe_pos, all_probes_only = probes(seq, extension_probe_length = 18, ligation_probe_length = 20, insert_size = 112)

In [10]:
len(all_probes)

733

In [11]:
final_probes = check_probes(all_probes_only)

In [12]:
len(final_probes)

258

In [17]:
# Converting list to dataframe 

def as_table(list_of_sequences):
    import pandas as pd
    data = pd.DataFrame(list_of_sequences, columns = ['Extension probe', 'Extension_probe_pos', 'Ligation probe', 'Extension_probe_pos'])
    return data

In [19]:
data = as_table(all_probes)

In [20]:
data

,Extension probe,Extension_probe_pos,Ligation probe,Extension_probe_pos
0,ATGTCACTGTATCGCCCT,start=0 end=18,TCGGTGTGTACGCGATGGAT,start=130 end=150
1,TGTCACTGTATCGCCCTC,start=1 end=19,CGGTGTGTACGCGATGGATA,start=131 end=151
2,GTCACTGTATCGCCCTCT,start=2 end=20,GGTGTGTACGCGATGGATAC,start=132 end=152
3,TCACTGTATCGCCCTCTA,start=3 end=21,GTGTGTACGCGATGGATACC,start=133 end=153
4,CACTGTATCGCCCTCTAG,start=4 end=22,TGTGTACGCGATGGATACCG,start=134 end=154
...,...,...,...,...
728,AAATGACTATGCCGTCGT,start=728 end=746,GGATTGGGCGTCAACGGGCA,start=858 end=878
729,AATGACTATGCCGTCGTC,start=729 end=747,GATTGGGCGTCAACGGGCAG,start=859 end=879
730,ATGACTATGCCGTCGTCT,start=730 end=748,ATTGGGCGTCAACGGGCAGT,start=860 end=880
731,TGACTATGCCGTCGTCTG,start=731 end=749,TTGGGCGTCAACGGGCAGTA,start=861 end=881


In [25]:
def check_probes(all_probes): #using all_probes as dataframe
    import Bio.SeqUtils 
    from Bio.SeqUtils import MeltingTemp as tm
    import pandas as pd
    
    
    # check for GC content of all_probes (expected range: 40-70%)
    for ext, lig in zip(data['Extension probe'], data['Ligation probe']):
        if Bio.SeqUtils.GC(ext) < 40 and Bio.SeqUtils.GC(ext) > 70 or Bio.SeqUtils.GC(lig) < 40 and Bio.SeqUtils.GC(lig) > 70:
            print(lig)
            # keep only those which follows the above criteria
            ind1 = int(data[data['Extension probe'] == ext].index.values)
            ind = data[data['Extension probe'] == lig].index.values
            data.drop([ind1], inplace=True)
            data.drop(ind, inplace=True)
           

    for ext, lig in zip(data['Extension probe'], data['Ligation probe']):
        if tm.Tm_Wallace(ext) < 60 or tm.Tm_Wallace(lig) < 60:
            # keep only those which follows the above criteria
            ind1 = int(data[data['Extension probe']== ext].index.values)
            ind = data[data['Extension probe']== lig].index.values
            data.drop([ind1], inplace=True)
            data.drop(ind, inplace=True)
                
    return data

## can include GC content and Tm in the dataframe and arrange in ascending order, to select best possible pairs of probes


In [26]:
check_probes(all_probes)

,Extension probe,Extension_probe_pos,Ligation probe,Extension_probe_pos
34,GTCTCTCATGGCCGCTGG,start=34 end=52,AACTGTAAGTTACCGCGCTG,start=164 end=184
35,TCTCTCATGGCCGCTGGC,start=35 end=53,ACTGTAAGTTACCGCGCTGA,start=165 end=185
36,CTCTCATGGCCGCTGGCT,start=36 end=54,CTGTAAGTTACCGCGCTGAG,start=166 end=186
37,TCTCATGGCCGCTGGCTG,start=37 end=55,TGTAAGTTACCGCGCTGAGG,start=167 end=187
38,CTCATGGCCGCTGGCTGG,start=38 end=56,GTAAGTTACCGCGCTGAGGA,start=168 end=188
...,...,...,...,...
678,GACTGGGCAGTCGGAGAC,start=678 end=696,ACAAGCACAGCGAGGCCGTC,start=808 end=828
699,ACCGGAACCTGCGGAGTG,start=699 end=717,TCGCCGCTGCGGCTAGACTC,start=829 end=849
700,CCGGAACCTGCGGAGTGT,start=700 end=718,CGCCGCTGCGGCTAGACTCG,start=830 end=850
709,GCGGAGTGTATGGCACGG,start=709 end=727,GGCTAGACTCGCGCTCGAGG,start=839 end=859


In [ ]:
def make_MIP(final_probes, linker):
    
    mips = []
    
    for (i,j) in final_probes:
        mip = j + linker + i
        mips.append(mip)
    return mips

    pass
        

In [ ]:
def open_folder(path):
    import os
    from Bio import SeqIO
    work_dir = os.chdir(path)
    
    seq = ''
    
    for file in os.listdir(work_dir):
        if file.endswith(".fna") or file.endswith(".fasta"):
            with open(file, 'r') as handle:
                handle.seek(0)
                
                for record in SeqIO.parse(handle, "fasta"):
                    
    pass
              